In [24]:
import random
import numpy as np
import tkinter as tk
from enum import Enum
from collections import deque


In [25]:


# --- ENUMS AND CLASSES ---
class CELL(Enum):
    EMPTY = 0
    CAR = 1
    PUBLIC_TRANSPORT = 2
    PEDESTRIAN = 3
    EMERGENCY = 4

class Intersection:
    def __init__(self, id, row, col):
        self.id = id
        self.row = row
        self.col = col
        self.green_light_interval = 30
        self.red_light_interval = 60
        self.yellow_light_interval = 5
        self.pedestrian_interval = 90
        self.priority = 'normal'  # normal, emergency, public_transport
        self.current_light = 'R'
        self.timer = self.red_light_interval

# --- INITIALIZE GRID AND INTERSECTIONS ---
class Grid:
    def __init__(self, size=(5, 5)):
        self.size = size
        self.grid = np.zeros(self.size, dtype=int)
        self.intersections = []
        self.setup_intersections()

    def setup_intersections(self):
        count = 1
        for r in range(self.size[0]):
            for c in range(self.size[1]):
                if count <= 20:
                    self.intersections.append(Intersection(f"I{count}", r, c))
                    count += 1

# --- CSP Functions ---
def adjust_for_priority(intersections):
    for inter in intersections:
        if inter.priority == 'emergency':
            inter.green_light_interval = max(45, inter.green_light_interval)
        elif inter.priority == 'public_transport':
            inter.green_light_interval = max(35, inter.green_light_interval)

def create_domains(intersections):
    domains = {}
    for inter in intersections:
        if inter.priority == 'emergency':
            domains[inter.id] = list(range(45, 61))
        elif inter.priority == 'public_transport':
            domains[inter.id] = list(range(35, 61))
        else:
            domains[inter.id] = list(range(15, 61))
    return domains

def is_consistent(x, y):
    return abs(x - y) > 5

def revise(domains, xi, xj):
    revised = False
    for x in domains[xi][:]:
        if not any(is_consistent(x, y) for y in domains[xj]):
            domains[xi].remove(x)
            revised = True
    return revised

def ac3(domains):
    queue = deque([(xi, xj) for xi in domains for xj in domains if xi != xj])
    while queue:
        xi, xj = queue.popleft()
        if revise(domains, xi, xj):
            if not domains[xi]:
                return False
            for xk in domains:
                if xk != xi and xk != xj:
                    queue.append((xk, xi))
    return True

def assign_values(intersections, domains):
    for inter in intersections:
        inter.green_light_interval = min(domains[inter.id])

def is_valid_assignment(intersections):
    for i in intersections:
        if i.green_light_interval < 15 or i.green_light_interval > 60:
            return False
        if i.red_light_interval < 30 or i.red_light_interval > 90:
            return False
        if i.yellow_light_interval < 3 or i.yellow_light_interval > 6:
            return False
    return True

def optimize_traffic(intersections):
    adjust_for_priority(intersections)
    domains = create_domains(intersections)
    if ac3(domains):
        assign_values(intersections, domains)
        if is_valid_assignment(intersections):
            print("Assignment valid after AC-3.")
        else:
            print("Assignment invalid even after AC-3.")
    else:
        print("AC-3 failed to make CSP consistent. No solution possible.")

# --- GUI CLASS ---
class TrafficGUI:
    def __init__(self, grid):
        self.grid = grid
        self.root = tk.Tk()
        self.root.title("Animated Smart City Traffic Manager")

        self.canvas = tk.Canvas(self.root, width=700, height=700, bg='white')
        self.canvas.pack()

        self.cell_size = 120
        self.light_radius = 12

        self.draw_static_grid()
        self.update_lights()
        self.root.mainloop()

    def draw_static_grid(self):
        for inter in self.grid.intersections:
            x = inter.col * self.cell_size + 100
            y = inter.row * self.cell_size + 100

            self.canvas.create_rectangle(x-20, y-20, x+20, y+20, fill='lightgray', outline='black', width=2)
            self.canvas.create_text(x, y-35, text=inter.id, font=('Arial', 10, 'bold'))

            inter.light_circle = self.canvas.create_oval(
                x - self.light_radius, y - self.light_radius,
                x + self.light_radius, y + self.light_radius,
                fill='red', outline='black'
            )
            inter.timer_text = self.canvas.create_text(x, y+35, text=f"{inter.timer}s", font=('Arial', 8))

    def update_lights(self):
        for inter in self.grid.intersections:
            inter.timer -= 1
            if inter.timer <= 0:
                if inter.current_light == 'G':
                    inter.current_light = 'Y'
                    inter.timer = inter.yellow_light_interval
                elif inter.current_light == 'Y':
                    inter.current_light = 'R'
                    inter.timer = inter.red_light_interval
                elif inter.current_light == 'R':
                    inter.current_light = 'G'
                    inter.timer = inter.green_light_interval

            color = {'R': 'red', 'G': 'green', 'Y': 'yellow'}[inter.current_light]
            self.canvas.itemconfig(inter.light_circle, fill=color)
            self.canvas.itemconfig(inter.timer_text, text=f"{inter.timer}s")

        self.root.after(1000, self.update_lights)

# --- MAIN EXECUTION ---
if __name__ == "__main__":
    city_grid = Grid()
    optimize_traffic(city_grid.intersections)
    gui = TrafficGUI(city_grid)


Assignment valid after AC-3.


In [4]:
import tkinter as tk
import numpy as np
import random
from enum import Enum
from collections import deque

# ---------------- ENUMS AND CONSTANTS ---------------- #
class CELL(Enum):
    EMPTY = 0
    CAR = 1
    PUBLIC_TRANSPORT = 2
    PEDESTRIAN = 3
    EMERGENCY = 4

# ---------------- INTERSECTION AND GRID ---------------- #
class Intersection:
    def __init__(self, id, row, col):
        self.id = id
        self.row = row
        self.col = col
        self.priority = 'normal'
        self.green_light_interval = 30
        self.red_light_interval = 60
        self.yellow_light_interval = 5
        self.pedestrian_interval = 90
        self.current_light = 'R'
        self.timer = self.red_light_interval

class Grid:
    def __init__(self, size=(4, 5)):
        self.size = size
        self.grid = np.zeros(size, dtype=int)
        self.intersections = self._init_intersections()

    def _init_intersections(self):
        intersections = []
        count = 1
        for r in range(self.size[0]):
            for c in range(self.size[1]):
                if count <= 20:
                    intersections.append(Intersection(f"I{count}", r, c))
                    count += 1
        return intersections

# ---------------- GUI FOR TRAFFIC LIGHTS ---------------- #
class TrafficGUI:
    def __init__(self, grid):
        self.grid = grid
        self.root = tk.Tk()
        self.root.title("Smart City Traffic Manager")
        self.canvas = tk.Canvas(self.root, width=800, height=700, bg='white')
        self.canvas.pack()
        self.cell_size = 120
        self.light_radius = 12
        self.vehicles = []
        self.vehicle_shapes = []
        self.draw_static_grid()
        self.spawn_vehicles()
        self.update_lights()
        self.animate_vehicles()
        self.root.mainloop()

    def draw_static_grid(self):
        for inter in self.grid.intersections:
            x = inter.col * self.cell_size + 100
            y = inter.row * self.cell_size + 100
            # Draw intersection box
            self.canvas.create_rectangle(x - 20, y - 20, x + 20, y + 20, fill='lightgray', outline='black', width=2)
            # Draw intersection ID
            self.canvas.create_text(x, y - 35, text=inter.id, font=('Arial', 10, 'bold'))
            # Draw traffic light
            inter.light_circle = self.canvas.create_oval(
                x - self.light_radius, y - self.light_radius,
                x + self.light_radius, y + self.light_radius, fill='red', outline='black'
            )
            inter.timer_text = self.canvas.create_text(x, y + 35, text=f"{inter.timer}s", font=('Arial', 8))

            # Draw road lines
            if inter.col < self.grid.size[1] - 1:
                self.canvas.create_line(x + 20, y, x + self.cell_size - 20, y, arrow=tk.LAST)
            if inter.row < self.grid.size[0] - 1:
                self.canvas.create_line(x, y + 20, x, y + self.cell_size - 20, arrow=tk.LAST)

    def spawn_vehicles(self):
        # Emergency vehicle on column 3
        self.vehicles.append({'type': 'emergency', 'row': 0, 'col': 3})
        # Car on row 0
        self.vehicles.append({'type': 'car', 'row': 0, 'col': 1})
        # Public transport on column 2
        self.vehicles.append({'type': 'bus', 'row': 0, 'col': 2})
        # Pedestrian at bottom-left
        self.vehicles.append({'type': 'pedestrian', 'row': 3, 'col': 0})

    def animate_vehicles(self):
        for shape in self.vehicle_shapes:
            self.canvas.delete(shape)
        self.vehicle_shapes.clear()

        for vehicle in self.vehicles:
            x = vehicle['col'] * self.cell_size + 100
            y = vehicle['row'] * self.cell_size + 100

            if vehicle['type'] == 'emergency':
                color = 'blue'
                direction = 'down'
            elif vehicle['type'] == 'car':
                color = 'black'
                direction = 'right'
            elif vehicle['type'] == 'bus':
                color = 'orange'
                direction = 'down'
            elif vehicle['type'] == 'pedestrian':
                color = 'green'
                direction = 'right'

            shape = self.canvas.create_oval(x - 10, y - 10, x + 10, y + 10, fill=color)
            self.vehicle_shapes.append(shape)

            if direction == 'down':
                vehicle['row'] += 0.05
            elif direction == 'right':
                vehicle['col'] += 0.05

        self.vehicles = [v for v in self.vehicles if v['row'] < self.grid.size[0] and v['col'] < self.grid.size[1]]
        self.root.after(100, self.animate_vehicles)

    def update_lights(self):
        for inter in self.grid.intersections:
            inter.timer -= 1
            if inter.timer <= 0:
                if inter.current_light == 'G':
                    inter.current_light = 'Y'
                    inter.timer = inter.yellow_light_interval
                elif inter.current_light == 'Y':
                    inter.current_light = 'R'
                    inter.timer = inter.red_light_interval
                elif inter.current_light == 'R':
                    inter.current_light = 'G'
                    inter.timer = inter.green_light_interval
            color = {'R': 'red', 'G': 'green', 'Y': 'yellow'}[inter.current_light]
            self.canvas.itemconfig(inter.light_circle, fill=color)
            self.canvas.itemconfig(inter.timer_text, text=f"{inter.timer}s")
        self.root.after(1000, self.update_lights)

# ---------------- MAIN EXECUTION ---------------- #
if __name__ == "__main__":
    city_grid = Grid(size=(4, 5))
    TrafficGUI(city_grid)


In [ ]:
import tkinter as tk
import numpy as np
import random
from enum import Enum
from collections import deque

# ---------------- ENUMS AND CONSTANTS ---------------- #
class CELL(Enum):
    EMPTY = 0
    CAR = 1
    PUBLIC_TRANSPORT = 2
    PEDESTRIAN = 3
    EMERGENCY = 4

CELL_SIZE = 100
ROWS, COLS = 4, 5
ENTITY_SIZE = 20

# ---------------- INTERSECTION AND GRID ---------------- #
class Intersection:
    def __init__(self, id, row, col):
        self.id = id
        self.row = row
        self.col = col
        self.priority = 'normal'
        self.green_light_interval = 30
        self.red_light_interval = 60
        self.yellow_light_interval = 5
        self.pedestrian_interval = 90
        self.current_light = 'R'
        self.timer = self.red_light_interval

class Grid:
    def __init__(self, size=(ROWS, COLS)):
        self.size = size
        self.grid = np.zeros(size, dtype=int)
        self.intersections = self._init_intersections()

    def _init_intersections(self):
        intersections = []
        count = 1
        for r in range(self.size[0]):
            for c in range(self.size[1]):
                intersections.append(Intersection(f"I{count}", r, c))
                count += 1
        return intersections

# ---------------- CSP FUNCTIONS ---------------- #
def adjust_priorities(intersections):
    for inter in intersections:
        if inter.priority == 'emergency':
            inter.green_light_interval = max(45, inter.green_light_interval)
        elif inter.priority == 'public_transport':
            inter.green_light_interval = max(35, inter.green_light_interval)

def create_domains(intersections):
    domains = {}
    for inter in intersections:
        if inter.priority == 'emergency':
            domains[inter.id] = list(range(45, 61))
        elif inter.priority == 'public_transport':
            domains[inter.id] = list(range(35, 61))
        else:
            domains[inter.id] = list(range(15, 61))
    return domains

def is_consistent(x, y):
    return abs(x - y) > 5

def revise(domains, xi, xj):
    revised = False
    for x in domains[xi][:]:
        if not any(is_consistent(x, y) for y in domains[xj]):
            domains[xi].remove(x)
            revised = True
    return revised

def ac3(domains):
    queue = deque([(xi, xj) for xi in domains for xj in domains if xi != xj])
    while queue:
        xi, xj = queue.popleft()
        if revise(domains, xi, xj):
            if not domains[xi]:
                return False
            for xk in domains:
                if xk != xi and xk != xj:
                    queue.append((xk, xi))
    return True

def assign_domains(intersections, domains):
    for inter in intersections:
        if domains[inter.id]:
            inter.green_light_interval = min(domains[inter.id])

def is_assignment_valid(intersections):
    for i in intersections:
        if not (15 <= i.green_light_interval <= 60): return False
        if not (30 <= i.red_light_interval <= 90): return False
        if not (3 <= i.yellow_light_interval <= 6): return False
    return True

def optimize_traffic(intersections):
    adjust_priorities(intersections)
    domains = create_domains(intersections)
    if ac3(domains):
        assign_domains(intersections, domains)
        return is_assignment_valid(intersections)
    return False

# ---------------- GUI SIMULATION ---------------- #
class SmartCityTrafficSim:
    def __init__(self, root, grid):
        self.root = root
        self.grid = grid
        self.root.title("Smart City Traffic Simulation")
        self.canvas = tk.Canvas(root, width=COLS * CELL_SIZE + 100, height=ROWS * CELL_SIZE + 100, bg='white')
        self.canvas.pack()

        self.intersections = {}
        self.vehicles = []
        self.pedestrians = []

        self.init_grid()
        self.spawn_entities()
        self.update_traffic_lights()
        self.animate_entities()

    def init_grid(self):
        for inter in self.grid.intersections:
            x = inter.col * CELL_SIZE + 60
            y = inter.row * CELL_SIZE + 60
            id_text = inter.id
            self.canvas.create_rectangle(x-15, y-15, x+15, y+15, fill='lightgray')
            self.canvas.create_text(x, y-25, text=id_text, font=('Arial', 10, 'bold'))
            light = self.canvas.create_oval(x-5, y-5, x+5, y+5, fill='red')
            self.intersections[id_text] = {'coords': (x, y), 'light': light, 'state': 'red'}

    def spawn_entities(self):
        car = self.canvas.create_rectangle(65, 65, 65+ENTITY_SIZE, 65+ENTITY_SIZE, fill='black')
        self.vehicles.append({'id': car, 'dx': 1, 'dy': 0})

        em = self.canvas.create_rectangle(260, 160, 260+ENTITY_SIZE, 160+ENTITY_SIZE, fill='blue')
        self.vehicles.append({'id': em, 'dx': 0, 'dy': 1})

        bus = self.canvas.create_rectangle(65, 265, 65+ENTITY_SIZE, 265+ENTITY_SIZE, fill='orange')
        self.vehicles.append({'id': bus, 'dx': 1, 'dy': 0})

        ped = self.canvas.create_oval(65, 365, 65+ENTITY_SIZE, 365+ENTITY_SIZE, fill='green')
        self.pedestrians.append({'id': ped, 'dx': 1, 'dy': 0})

    def update_traffic_lights(self):
        for inter in self.intersections.values():
            current_color = inter['state']
            next_color = 'green' if current_color == 'red' else 'red'
            inter['state'] = next_color
            self.canvas.itemconfig(inter['light'], fill=next_color)
        self.root.after(3000, self.update_traffic_lights)

    def animate_entities(self):
        for vehicle in self.vehicles:
            self.canvas.move(vehicle['id'], vehicle['dx'] * 2, vehicle['dy'] * 2)
        for ped in self.pedestrians:
            self.canvas.move(ped['id'], ped['dx'], ped['dy'])
        self.root.after(100, self.animate_entities)

# ---------------- MAIN EXECUTION ---------------- #
if __name__ == '__main__':
    city_grid = Grid()
    if optimize_traffic(city_grid.intersections):
        print("Traffic optimized successfully.")
    else:
        print("Failed to optimize traffic.")
    root = tk.Tk()
    app = SmartCityTrafficSim(root, city_grid)
    root.mainloop()


Traffic optimized successfully.
